In [2]:
import numpy as np
import pandas as pd 
from datetime import datetime

%matplotlib inline
import matplotlib
%config InlineBackend.figure_format = 'jpg'
matplotlib.figure.Figure.__repr__ = lambda self: (
    f"<{self.__class__.__name__} size {self.bbox.size[0]:g}"
    f"x{self.bbox.size[1]:g} with {len(self.axes)} Axes>")
import matplotlib.pyplot as plt

Branje podatkov

In [ ]:
netflix = pd.read_csv("Data\\netflix_titles.csv")

movies = pd.read_csv("Data\\akas.csv")
ratings = pd.read_csv("Data\\ratings.csv")
names = pd.read_csv("Data\\names_basic.csv")

crew=pd.read_csv("Data\\crew.csv")
crew.rename(columns = {'tconst':'titleId'}, inplace = True) 

director_names_and_stats=pd.read_csv("Data\\namebasics.csv")
director_names_and_stats.rename(columns = {'tconst':'titleId'}, inplace = True) 

In [ ]:
ratings = ratings.rename(columns = {'tconst':'titleId'})
originals = movies[movies["types"] == "original"]
originals = originals[["titleId", "title"]]
originals = originals.merge(ratings, on="titleId")

projekt = originals.merge(crew[["titleId","directors"]], on="titleId")

originals = originals.rename(columns = {'tconst':'title'})
netflix = netflix.merge(originals, on="title")


In [ ]:
def najdi(table):
    if table.empty:
        return table
    else:
        return table.iloc[table["numVotes"].argmax()]

def getMovieDirID(MovieName):
    for index, row in projekt.iterrows():
        if (projekt.loc[index, row["titleId"]] == MovieName):
            return projekt.loc[index, row["directors"]]

def getDirector(dirID):
    df=director_names_and_stats[["nconst","primaryName"]]
    for index, row in df.iterrows():     
        if(df.loc[index, row["nconst"]] == dirID):
            return df.loc[index,"primaryName"]
            

In [ ]:
netflix1=netflix.fillna(0)

In [ ]:
i = 0
for idx, naslov in enumerate(netflix1["title"]):
    i += 1
    table = originals[originals["title"] == naslov]
    if table.empty:
        pass
    else:
        a = najdi(originals[originals["title"] == naslov])
        netflix1.loc[idx, "averageRating"] = a["averageRating"]
        netflix1.loc[idx, "numVotes"] = a["numVotes"]

Preberemo imdb bazo za mankajoce podatke

In [ ]:
from imdb import IMDb
from imdb import IMDbDataAccessError

movs = IMDb()
i = 0

for idx, naslov in enumerate(netflix1["title"]):
    i += 1
    if netflix1.loc[idx, "director"] == "0":
        m = movs.search_movie(naslov)
        if len(m) > 0:
            try:
                data = movs.get_movie(m[0].movieID)
                netflix1.loc[idx, "director"] = data["director"][0]['name']
            except KeyError as e:
                netflix1.loc[idx, "director"] = "-1"
            except (URLError, IMDbDataAccessErro) as e:
                print(data)
    if i%100 == 0:
        print(i)

In [15]:
netflix1 = pd.read_csv("Netflix_all.csv")
#netflix1['date_added'].to_csv("D:\\OneDrive\\PhenixZeroPrime\FRI\\2. Letnik 2. Semester\\Podatkovno Rudarjenje\\PR_Projektna\\PR19KZMBG\\datumi.csv", index=False)


Popravilo datumov

In [4]:
dic = {'January':'1', 'February':'2', 'March':'3', 'April':'4', 'May':'5', 'June':'6', 'July':'7', 'August':'8', 'September':'9', 'October':'10', 'November':'11', 'December':'12', '':'0'}
netflix1['month_added'] = 0
netflix1['year_added'] = 0
for index, row in netflix1.iterrows():
    if(row['date_added'] != "0"):  
        monthDay_year = row['date_added'].split(", ") #[September 9, 2019]
        month_day = monthDay_year[0].split(" ") #[September, 9]
        if(len(month_day) == 2): #Če je split enak: month_day == [September, 9]
            netflix1.loc[index, "date_added"] = "%s/%s/%s" % (dic[month_day[0]], month_day[1], monthDay_year[1])
            try:
                netflix1.loc[index, 'date_added'] = pd.to_datetime(netflix1.loc[index, 'date_added'], format="%m/%d/%Y")
            except (TypeError, ValueError) as e:
                print(netflix1.loc[index, 'date_added'])
            netflix1.loc[index, "month_added"]= month_day[0]
            netflix1.loc[index, "year_added"]= monthDay_year[1]

        elif(len(month_day) == 3): #Če je split napačen: month_day == ['', September, 9]
            netflix1.loc[index, "date_added"] = "%s/%s/%s" % (dic[month_day[1]], month_day[2], monthDay_year[1])
            try:
                netflix1.loc[index, 'date_added'] = pd.to_datetime(netflix1.loc[index, 'date_added'], format="%m/%d/%Y")
            except (TypeError, ValueError) as e:
                print(netflix1.loc[index, 'date_added'])
                print(month_day[1]) 
            netflix1.loc[index, "month_added"]= month_day[1]
            netflix1.loc[index, "year_added"]= monthDay_year[1]

In [10]:
netflixWithoutMissingDate = netflix1[netflix1['date_added'] != '0']
netflixWithoutMissingDate = netflixWithoutMissingDate.astype({'year_added': 'int64'})
netflixWithoutMissingDate['date_added']= pd.to_datetime(netflixWithoutMissingDate['date_added'], format="%d/%m/%Y") 

In [19]:
netflix1 = netflix1.astype({'year_added': 'int64'})
netflix1['date_added']= pd.to_datetime(netflix1['date_added'], format="%Y/%m/%d", errors='coerce') 
netflix1.tail(29)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,averageRating,numVotes,month_added,year_added
6205,80036747,TV Show,Pororo - The Little Penguin,-1,0,South Korea,2019-04-19,2013,TV-Y,2 Seasons,"Kids' TV, Korean TV Shows","On a tiny island, Pororo the penguin has fun a...",5.7,41.0,April,2019
6206,80173174,TV Show,Samantha!,-1,"Emmanuelle Araújo, Douglas Silva, Sabrina Nona...",Brazil,2019-04-19,2019,TV-MA,2 Seasons,"International TV Shows, TV Comedies","A child star in the '80s, Samantha clings to t...",6.5,428.0,April,2019
6207,80190407,TV Show,Murderous Affairs,-1,0,United States,2018-04-17,2017,TV-14,3 Seasons,"Crime TV Shows, Docuseries","Mixing interviews with dramatic re-enactments,...",6.8,153.0,April,2018
6208,70227189,TV Show,Lost Girl,-1,"Anna Silk, Kris Holden-Ried, Ksenia Solo, Rich...",Canada,2016-04-17,2015,TV-14,5 Seasons,"TV Dramas, TV Horror, TV Mysteries",Discovering she's a succubus who sustains hers...,7.7,27777.0,April,2016
6209,70264078,TV Show,Mr. Young,-1,"Brendan Meyer, Matreya Fedor, Gig Morton, Kurt...",Canada,2019-04-16,2013,TV-G,2 Seasons,"Kids' TV, TV Comedies","After Adam graduates from college at age 14, h...",6.1,1138.0,April,2019
6210,80239700,TV Show,Psiconautas,Pedro Rivero,"Guillermo Toledo, Gabriel Goity, Florencia Peñ...",Argentina,2018-04-15,2016,TV-MA,2 Seasons,"International TV Shows, Spanish-Language TV Sh...",A Spanish con man masquerades as a therapist a...,7.0,101.0,April,2018
6211,80231523,TV Show,The Minimighty Kids,-1,0,France,2018-04-15,2012,TV-G,2 Seasons,"Kids' TV, TV Comedies","Some have big feet or a sniffly nose, others a...",5.8,26.0,April,2018
6212,80126877,TV Show,Filinta,-1,"Onur Tuna, Serhat Tutumluer, Mehmet Özgür, Naz...",Turkey,2017-04-15,2015,TV-14,2 Seasons,"Crime TV Shows, International TV Shows, TV Act...","In 19th-century Istanbul, a young police offic...",6.7,2393.0,April,2017
6213,80126599,TV Show,Leyla and Mecnun,Onur Ünlü,"Ali Atay, Melis Birkan, Serkan Keskin, Ahmet M...",Turkey,2017-04-15,2014,TV-PG,3 Seasons,"International TV Shows, Romantic TV Shows, TV ...",Destiny brings Mecnun and Leyla together as ne...,9.1,85391.0,April,2017
6214,80049872,TV Show,Chelsea,-1,0,United States,2017-04-14,2017,TV-MA,2 Seasons,"Stand-Up Comedy & Talk Shows, TV Comedies","It's not her first talk show, but it is a firs...",6.1,1882.0,April,2017


tu nevemo kaj nrdi

In [20]:
netflix1.to_csv("D:\\OneDrive\\PhenixZeroPrime\FRI\\2. Letnik 2. Semester\\Podatkovno Rudarjenje\\PR_Projektna\\PR19KZMBG\\Netflix_all.csv", index=False)

In [ ]:
#netflix1 = netflix1.sort_values(by='date_added', ascending=False)
#netflix1 = pd.to_datetime(netflix1['date_added'], dayfirst=True)
#netflix1['date_added'] = pd.to_datetime(netflix1['date_added']).dt.year
#netflix1['Year'] = pd.DatetimeIndex(netflix1['date_added']).month 
#tv_show = df[df["type"] == "TV Show"]
#movies = df[df["type"] == "Movie"]
for index, row in netflix1.iterrows():
    if(row['date_added'] != '0'):
        aaa = row['date_added']
        new = aaa.split("-")
        netflix1.loc[index, "month_added"]= new[1]
        netflix1[index, "year_added"]= new[2] 
        